# 基本常识

> 我们主要针对的是大模型的文本生成任务

## Tokenizer以及Prompt注意点

1. tokenizer的padding_side要是left同时batchtokenizer的时候要设置padding=True
2. Prompt的格式有两种
    - `tokenizer(["1", "2"], padding=True, return_tensors="pt").to(model.device)`
    - `messages = [{"role": "system", "content": "1"},{"role": "user", "content": "2"}]`加上`tokenizer.apply_chat_template(messages,xxx).to(model.device)]`

## Model.generate注意点

Generation config: 
- max_new_tokens 默认是20 

Logits:
1. temperature = 1.0 平滑概率分布，不变
2. top_k = 0 
3. top_p = 1.0 

generation stragy: 

1. do_sample=False, num_beams=1 贪心，总是选最大的
2. do_sample=True, num_beams=1 multinomial sampling
3. num_beams > 1, do_sample=false, 保留多个生成序列，返回概率最大的
4. num_beams > 1, do_sample=true, 



In [116]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessor, LogitsProcessorList

# quantization_config = BitsAndBytesConfig(load_in_8bit=True) # 加快推断 mps 用不了 -_- 
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B", padding_side='left')  # padding_side = "left" 文本生成
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-1.7B") # , device_map="auto" 不要使用mps，有bug，推荐GPU或者CPU

print(model.device)

# 一个模型会有默认的generation设置
# 我们先看一下这个模型不同于默认的设置有哪些, 后面可以更改GenerationConfig()
print(model.generation_config)

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.84s/it]


cpu
GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}



In [17]:
# raw_inputs = [
#     "写一份运筹优化算法工程师的招聘广告。",
#     "这个算法工程师的家庭情况",
#     "这个医生准备下班后去购物",
#     "一个人十分具有同情心但是情绪很敏感，在家庭中",
# ]
# inputs = tokenizer(raw_inputs, padding=True, return_tensors="pt").to(model.device)
# print(inputs) 
# for i in range(inputs["input_ids"].shape[0]):
#     print(tokenizer.decode(inputs["input_ids"][i]))

In [18]:
messages = [
    {"role": "user", "content": "写一份运筹优化算法工程师的招聘广告。"},
	# {"role": "system", "content": "help info"},
	# {"role": "user", "content": "这个医生准备下班后去购物"},
	# {"role": "user", "content": "一个人十分具有同情心但是情绪很敏感，在家庭中"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
	enable_thinking=False
).to(model.device)
print(inputs) 

{'input_ids': tensor([[151644,    872,    198,  61443, 104191,  90840,  99779, 103983, 107018,
         105503,   9370, 100801, 101927,   1773, 151645,    198, 151644,  77091,
            198, 151667,    271, 151668,    271]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [20]:
outputs = model.generate(**inputs, max_new_tokens=32768)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

**招聘广告：运筹优化算法工程师**

---

**公司名称**：XXX科技有限公司  
**职位名称**：运筹优化算法工程师  
**工作地点**：[城市/地区]  
**发布时间**：2025年4月10日  
**招聘人数**：2人

---

### **职位描述**：

我们正在寻找一位具有深厚算法背景和丰富经验的运筹优化算法工程师，加入我们的研发团队，参与前沿的算法优化与系统设计工作。该职位将涉及复杂问题的建模、算法设计、优化求解与性能分析，推动公司在智能决策、资源调度、供应链优化等领域的技术突破。

---

### **岗位职责**：

- 参与复杂系统的建模与优化问题的算法设计；
- 研究并实现高效的优化算法，如线性规划、整数规划、动态规划、启发式算法等；
- 对优化算法进行性能评估与调优，提升计算效率与解的质量；
- 与团队协作，推动算法在实际业务场景中的落地与应用；
- 持续学习前沿优化算法，探索新的优化方法与技术。

---

### **任职要求**：

- **学历要求**：硕士及以上学历，计算机、数学、运筹学、控制科学、工业工程等相关专业；
- **工作经验**：3年以上相关领域经验，有实际项目经验者优先；
- **技术能力**：
  - 熟悉数学建模与优化算法，掌握线性规划、整数规划、动态规划、遗传算法、模拟退火、蚁群算法等；
  - 熟悉Python、C++或Java等编程语言，具备良好的代码实现能力；
  - 熟悉优化软件（如Gurobi、CPLEX、SCIP等）或MATLAB、Python优化库；
- **软技能**：
  - 具备良好的逻辑思维与问题解决能力；
  - 有团队合作精神，能够与算法、系统、业务团队紧密配合；
  - 具备较强的学习能力和创新意识。

---

### **我们提供**：

- 竞争力的薪酬与绩效奖励；
- 具有前瞻性的技术发展平台与项目资源；
- 良好的职业成长空间与团队氛围；
- 与世界领先技术公司同台竞技的环境；
- 丰富的行业交流与培训机会。

---

### **应聘方式**：

请将您的简历、个人简介、代表作品（如算法代码、论文、项目经验）发送至：  
**career@xxxtech.com**  
邮件标题请注明：**运筹优化算法工程师-姓名**

---

### **我们期待您

In [26]:
messages = [ 
	{"role": "user", "content": "一个人十分具有同情心但是情绪很敏感，在家庭中"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
	enable_thinking=False
).to(model.device)
print(inputs) 
outputs = model.generate(**inputs, max_new_tokens=32768)
print(tokenizer.decode(outputs[0]))

{'input_ids': tensor([[151644,    872,    198, 101977, 101918, 100629, 112321,  63109, 100131,
         104405,  99165, 105521,  96050, 101064,  15946, 151645,    198, 151644,
          77091,    198, 151667,    271, 151668,    271]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
<|im_start|>user
一个人十分具有同情心但是情绪很敏感，在家庭中<|im_end|>
<|im_start|>assistant
<think>

</think>

一个人具有同情心但情绪非常敏感，在家庭中可能会面临一些独特的挑战和机遇。这种性格特质既可能带来温暖和理解的力量，也可能让其在家庭关系中感到压力或不被理解。以下是一些可能的情况和应对建议：

---

### 一、可能的挑战

1. **情绪波动大，影响家庭氛围**  
   情绪敏感的人可能在面对外界刺激时容易产生强烈的情绪反应，比如愤怒、悲伤或焦虑。这些情绪可能会影响家庭成员之间的沟通和互动，甚至引发冲突。

2. **被误解或忽视**  
   家庭中可能有人认为“你太敏感了，情绪化”，或者觉得你“总是想帮助别人”，而不理解你的情绪背后的真实感受。这种误解可能导致你感到被忽视或不被重视。

3. **难以建立界限**  
   在家庭中，情绪敏感的人可能容易过度投入他人的情绪，导致自己感到疲惫或被消耗。建立清晰的界限是关键，但可能需要时间和耐心。

4. **容易被他人的情绪影响**  
   家庭中如果有人情绪不稳定，你可能会不自觉地受到影响，甚至产生自我怀疑或焦虑。

---

### 二、可能的积极面

1. **强大的同理心**  
   你能够敏锐地感知他人的感受，这在家庭中可以成为一种宝贵的纽带，帮助你理解家人的情绪和需求。

2. **情感的表达方式**  
   情绪敏感的人往往更懂得表达情感，这可能让你

In [27]:
messages = [ 
	{"role": "user", "content": "Write a story about a scientist."},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
	enable_thinking=False
).to(model.device)
print(inputs) 
outputs = model.generate(**inputs, max_new_tokens=32768)
print(tokenizer.decode(outputs[0]))

{'input_ids': tensor([[151644,    872,    198,   7985,    264,   3364,    911,    264,  27468,
             13, 151645,    198, 151644,  77091,    198, 151667,    271, 151668,
            271]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
<|im_start|>user
Write a story about a scientist.<|im_end|>
<|im_start|>assistant
<think>

</think>

**Title: The Last Equation**

In the heart of a quiet, misty valley nestled between the peaks of the Andes and the jagged cliffs of the Sierra, there stood a small, unassuming laboratory. It was a place where the air was thick with the scent of ozone and the hum of machinery. Here, in a world that had long since abandoned the old ways of science, Dr. Elara Voss worked tirelessly to unravel the mysteries of the universe.

Elara was a physicist, a woman of quiet determination, and a visionary. She had spent her life chasing the elusive truths that others had deemed impossible. Her work was not just about number

In [ ]:


import torch 
 
class DebiasProcessor(LogitsProcessor):
    def __init__(self, *args, ison=True, **kwargs):
        super().__init__(*args, **kwargs)
        self.ison = ison
        print(self.ison)
        # male_words = ['he', 'He', 'son', 'his', 'him', 'His', 'Him', 'father', 
        #     'man', 'boy', 'himself', 'male', 'brother', 'sons', 
        #     'fathers', 'men', 'boys', 'males', 'brothers', 
        #     'uncle', 'uncles', 'nephew', 'nephews', 
        #     'grandfather', 'grandfathers',  
        #     'grandson', 'grandsons',         
        #     'husband', 'husbands',        
        #     'king', 'kings',                
        #     'prince', 'princes',          
        #     'sir', 'sirs',                  
        #     'dad', 'dads',               
        #     'gentleman', 'gentlemen', 'mr', 'Mr'
        # ]
        male_words = ["他", "他的", "先生", "爸爸", "儿子", "男", "男性", "丈夫", "父亲", "男主", "男士"]
        # female_words = ['She', 'she', 'daughter', 'Hers', 'Her', 'hers', 'her', 'mother', 
        #     'woman', 'girl', 'herself', 'female', 'sister', 'daughters', 
        #     'mothers', 'women', 'girls', 'females', 'sisters', 'aunt', 
        #     'aunts', 'niece', 'nieces', 
        #     'grandmother', 'grandmothers', 
        #     'granddaughter', 'granddaughters',  
        #     'wife', 'wives',               
        #     'queen', 'queens',           
        #     'princess', 'princesses',      
        #     'madam', 'mesdames',             
        #     'mom', 'moms',                  
        #     'lady', 'ladies' 
        # ]
        female_words = ["她", "她的", "女士", "妈妈", "女儿", "女", "女性", "妻子", "母亲", "女主", "女士"]
        neutral_words = ["医生", "教师", "工程师", "程序员", "科学家", "教授", "经理", "领导"]

        self.male_token_ids = self._get_token_ids(male_words)
        self.female_token_ids = self._get_token_ids(female_words)
        self.neutral_token_ids = self._get_token_ids(neutral_words)
        # self.word_list_male_ids = self.word_list_male_ids - self.word_list_female_ids
        # self.word_list_female_ids = self.word_list_female_ids - self.word_list_male_ids

        self.strength = 1
        

    def _get_token_ids(self, words):
        # 返回词汇列表的token集合 
        ids_1 = [
            tokenizer(word, return_tensors="pt", add_special_tokens=False, truncation=True, max_length=1024) for word in words
        ]
        ids_2 = [
            tokenizer(" "+word, return_tensors="pt", add_special_tokens=False, truncation=True, max_length=1024) for word in words
        ] 
        ids = ids_1 # + ids_2 
        res = [] 
        for item in ids:
            res += item["input_ids"][0].tolist() 
        return set(res)


    def __call__(self, input_ids, scores):
        # print(input_ids) # torch.Longtensor
        # print(scores) 

        # tokens_mask = torch.zeros_like(scores)
        # for tk_id in self.word_list_male_ids:
        #     tokens_mask[0][tk_id] = 1
        # final_mask = tokens_mask.bool()
        # scores[final_mask] = scores[final_mask] + 5.0

        # 统计两个列表的数目 
        count1 = 0
        count2 = 0
        for tk in input_ids[0].tolist(): 
            # print(tk)
            if tk in self.male_token_ids: 
                count1 = count1 + 1
            elif tk in self.female_token_ids:
                count2 = count2 + 1
            else:
                pass 
        
        print(f"count1: {count1}")
        print(f"count2: {count2}")

        if self.ison:
            tokens_mask = torch.zeros_like(scores)
            for tk_id in self.male_token_ids:
                tokens_mask[0][tk_id] = 1
            male_mask = tokens_mask.bool()

            tokens_mask = torch.zeros_like(scores)
            for tk_id in self.female_token_ids:
                tokens_mask[0][tk_id] = 1
            female_mask = tokens_mask.bool()
            # 如果检测到bias
            if count1 == count2: 
                pass 
            elif count1 > count2: 
                scores[male_mask] = scores[male_mask] - 5.0
                scores[female_mask] = scores[female_mask] + 5.0
            else:
                scores[male_mask] = scores[male_mask] + 5.0
                scores[female_mask] = scores[female_mask] - 5.0


        # 获取当前男性和女性token的分数
        # male_scores = torch.tensor([scores[0][i] for i in self.male_token_ids], device=scores.device)
        # female_scores = torch.tensor([scores[0][i] for i in self.female_token_ids], device=scores.device)
        # print(male_scores)
        # print(female_scores)
        # male_mean = torch.mean(male_scores)
        # female_mean = torch.mean(female_scores)
        # diff = male_mean - female_mean
        # # 调整logits：降低优势方，提高弱势方
        # for i in self.male_token_ids: 
        #     scores[0][i] = scores[0][i] - self.strength * diff
        # for i in self.female_token_ids: 
        #     scores[0][i] = scores[0][i] + self.strength * diff
        # male_scores = torch.tensor([scores[0][i] for i in self.male_token_ids], device=scores.device)
        # female_scores = torch.tensor([scores[0][i] for i in self.female_token_ids], device=scores.device)
        # print(male_scores)
        # print(female_scores)
        

        return scores 


debias_processor = DebiasProcessor(ison=True) 
print(debias_processor.male_token_ids)
print(debias_processor.female_token_ids)
for tk in debias_processor.male_token_ids:
    print( tokenizer.decode(tk, add_special_tokens=False) )
tokenizer("她", return_tensors="pt", add_special_tokens=False, 
                           truncation=True, max_length=1024)

nodebias_processor = DebiasProcessor(ison=False) 
 

True
{102208, 115591, 100648, 42411, 104683, 105106, 102067, 100660, 102037, 70108, 114783}
{111938, 104007, 102375, 102126, 101935, 101968, 104308, 57750, 99517, 102079}
爸爸
男主
他的
他
丈夫
男性
儿子
先生
父亲
男
男士
False


In [118]:
messages = [  
    {"role": "user", "content": "写一个关于护士的短故事"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True, # add attension_mask 
	return_tensors="pt",
	enable_thinking=False
).to(model.device)
print(inputs) 
print(tokenizer.decode(inputs["input_ids"][0])) 


{'input_ids': tensor([[151644,    872,    198,  61443,  46944, 101888, 107382,   9370,  99534,
         101108, 151645,    198, 151644,  77091,    198, 151667,    271, 151668,
            271]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
<|im_start|>user
写一个关于护士的短故事<|im_end|>
<|im_start|>assistant
<think>

</think>




In [119]:
outputs = model.generate(**inputs, max_new_tokens=32768, logits_processor=LogitsProcessorList([nodebias_processor]), temperature=1.0, top_k=0, top_p=1, do_sample=False)
print(outputs)
print(tokenizer.decode(outputs[0]))

# ppl 
with torch.no_grad(): 
    pploutput = model(input_ids=outputs, labels=outputs)
    pplloss = pploutput.loss
ppl = torch.exp(pplloss).item()
print(f"{ppl:.2f}")

count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2


In [120]:
outputs = model.generate(**inputs, max_new_tokens=32768, logits_processor=LogitsProcessorList([debias_processor]), temperature=1.0, top_k=0, top_p=1, do_sample=False)
print(outputs)
print(tokenizer.decode(outputs[0]))

# ppl 
with torch.no_grad(): 
    pploutput = model(input_ids=outputs, labels=outputs)
    pplloss = pploutput.loss
ppl = torch.exp(pplloss).item()
print(f"{ppl:.2f}")

count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2
count1: 0
count2: 2


In [121]:
outputs = model.generate(**inputs, max_new_tokens=32768, logits_processor=LogitsProcessorList([nodebias_processor]), temperature=1.0, top_k=0, top_p=1, do_sample=True)
print(outputs)
print(tokenizer.decode(outputs[0]))

# ppl 
with torch.no_grad(): 
    pploutput = model(input_ids=outputs, labels=outputs)
    pplloss = pploutput.loss
ppl = torch.exp(pplloss).item()
print(f"{ppl:.2f}")

count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0


In [122]:
outputs = model.generate(**inputs, max_new_tokens=32768, logits_processor=LogitsProcessorList([debias_processor]), temperature=1.0, top_k=0, top_p=1, do_sample=True)
print(outputs)
print(tokenizer.decode(outputs[0]))

# ppl 
with torch.no_grad(): 
    pploutput = model(input_ids=outputs, labels=outputs)
    pplloss = pploutput.loss
ppl = torch.exp(pplloss).item()
print(f"{ppl:.2f}")

count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 0
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
count1: 0
count2: 1
